In [1]:
import tensorflow as tf
import pandas as pd
from sdv.single_table import CTGANSynthesizer


In [24]:
path = "C:\\Users\\Lisa\\Desktop\\FH\\GAI\\Pokemon\\data\\preprocessed_pokemon_with_Evolution.csv"
df = pd.read_csv(path, encoding= "UTF-8")
df.head()

,Unnamed: 0,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Evolution
0,0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,1.0
1,1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,2.0
2,2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,3.0
3,3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,5.0
4,4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,1.0


In [25]:
from sdv.datasets.local import load_csvs

# assume that my_folder contains a CSV file named 'guests.csv'
datasets = load_csvs(
    folder_name='data/',
    read_csv_parameters={
        'skipinitialspace': True,
        'encoding': 'utf-8'
    })

# the data is available under the file name
data = datasets['Pokemon']

In [26]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)


In [27]:
from sdv.single_table import GaussianCopulaSynthesizer

# Step 1: Create the synthesizer
synthesizer = GaussianCopulaSynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(df)

# Step 3: Generate synthetic data
synthetic_data_gaussian = synthesizer.sample(num_rows=100)

c:\Users\Lisa\anaconda3\envs\FH\Lib\site-packages\sdv\single_table\base.py:81: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



In [28]:
from sdv.single_table import CTGANSynthesizer
synthesizer = CTGANSynthesizer(
    metadata, # required
    enforce_rounding=False,
    epochs=500,
    verbose=True
)

In [37]:
synthesizer.fit(df)

synthetic_data_ctgan = synthesizer.sample(num_rows=400)

c:\Users\Lisa\anaconda3\envs\FH\Lib\site-packages\sdv\single_table\ctgan.py:24: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

Gen. (-1.00) | Discrim. (0.00): 100%|██████████| 500/500 [02:12<00:00,  3.78it/s] 


In [30]:
synthetic_data_ctgan.head()

,Unnamed: 0,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Evolution
0,0,sdv-pii-64jr8,Water,Psychic,26,125,176,116,77,70,4,False,2.0
1,1,sdv-pii-xnuld,Fairy,Dragon,135,161,51,19,93,5,2,False,1.0
2,2,sdv-pii-23pgo,Ground,Electric,52,100,84,30,44,17,4,False,1.0
3,3,sdv-pii-8mult,Electric,Water,91,190,79,33,74,18,3,False,2.0
4,4,sdv-pii-pjbi7,Fire,Ground,56,113,115,88,52,48,3,True,2.0


In [38]:
synthetic_data_ctgan.to_csv("CTGAN_Pokemon.csv", encoding="UTF-8", index=False)
synthetic_data_gaussian.to_csv("Gaussian_Pokemon.csv", encoding="UTF-8", index=False)

In [31]:
# Eval not really working
from sdmetrics.reports.single_table import QualityReport

report = QualityReport()

In [32]:
metadata_dict = {
    "columns": {
        "#": {
            "sdtype": "numerical"
        },
        "Name": {
            "sdtype": "id"
        },
        "Type 1": {
            "sdtype": "categorical"
        },
        "Type 2": {
            "sdtype": "categorical"
        },
        "Total": {
            "sdtype": "numerical"
        },
        "HP": {
            "sdtype": "numerical"
        },
        "Attack": {
            "sdtype": "numerical"
        },
        "Defense": {
            "sdtype": "numerical"
        },
        "Sp. Atk": {
            "sdtype": "numerical"
        },
        "Sp. Def": {
            "sdtype": "numerical"
        },
        "Speed": {
            "sdtype": "numerical"
        },
        "Generation": {
            "sdtype": "categorical"
        },
        "Legendary": {
            "sdtype": "categorical"
        }
    },
    "primary_key": "Name",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}


In [33]:
report.generate(df, synthetic_data_ctgan, metadata_dict)

ValueError: The metadata does not match the data. The following columns are missing in the real/synthetic data or in the metadata: #, Evolution, Total, Unnamed: 0

In [ ]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,#,KSComplement,0.602500
1,Type 1,TVComplement,0.358750
2,Type 2,TVComplement,0.241546
3,Total,KSComplement,0.785000
4,HP,KSComplement,0.331250
5,Attack,KSComplement,0.531250
6,Defense,KSComplement,0.417500
7,Sp. Atk,KSComplement,0.818750
8,Sp. Def,KSComplement,0.583750
9,Speed,KSComplement,0.802500
